In [ ]:
import cv2
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error, mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr
import torch
import torchvision.transforms as transforms
from scipy.ndimage import convolve
import lpips
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from torchvision.models.inception import inception_v3
from torchvision import transforms
from scipy.linalg import sqrtm

## Calculate metrics

In [ ]:
original_image = Image.open("C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\results\\dualseq-bt_rg-v1\\test_latest\\images\\PANC-1_14_real_B.png").convert('L')
generated_image = Image.open('C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\results\\dualseq-bt_rg-v1_recur-2\\dualseq-bt_rg-v1_sketch\\test_latest\\images\\step14_generated_fake_B.png').convert('L')

original_array = np.array(original_image)
generated_array = np.array(generated_image)

#### Run only once

In [ ]:
# Inicializar el modelo LPIPS
loss_fn = lpips.LPIPS(net='vgg')

In [ ]:
def calculate_metrics(original_image, generated_image):
    # Convertir imágenes PIL a matrices NumPy
    original_array = np.array(original_image)
    generated_array = np.array(generated_image)
    
    ## Calcular PSNR
    psnr_value = psnr(original_array, generated_array)
    ## Calcular SSIM
    ssim_value = ssim(original_array, generated_array, win_size=7)
    ## Calcular MSE
    mse = mean_squared_error(original_array.flatten(), generated_array.flatten())
    ## Calcular MAE
    mae = mean_absolute_error(original_array.flatten(), generated_array.flatten())
    
    ## Calcular LPIPS
    # Transformaciones necesarias para las imágenes
    transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()]) # Modificar en función tal tamaño empleado
    # Aplicar transformaciones a las imágenes
    original_img = transform(original_image)
    generated_img = transform(generated_image)
    # Preparar las imágenes para la entrada en la métrica LPIPS
    original_img = original_img.unsqueeze(0)
    generated_img = generated_img.unsqueeze(0)
    
    # Calcular el LPIPS entre las imágenes
    lpips_value = loss_fn(original_img, generated_img).item()

    ## Calcular correlación de Pearson
    pearson_corr = np.corrcoef(original_array.flatten(), generated_array.flatten())[0, 1]

    ## Calcular entropia cruzada
    # Calcular histogramas normalizados de las imágenes
    hist_original, _ = np.histogram(original_array.flatten(), bins=256, range=[0, 256], density=True)
    hist_generated, _ = np.histogram(generated_array.flatten(), bins=256, range=[0, 256], density=True)
    epsilon = 1e-10
    hist_org = np.maximum(hist_original, epsilon)
    hist_gen = np.maximum(hist_generated, epsilon)
    cross_entropy = -np.sum(hist_org * np.log(hist_gen))
    
    ## Calcular la diferencia de histograma
    # Calcular histogramas
    hist_original_diff, _ = np.histogram(original_array.flatten(), bins=256, range=[0,256])
    hist_generated_diff, _ = np.histogram(generated_array.flatten(), bins=256, range=[0,256])
    # Normalizar histogramas
    hist_original_norm = hist_original_diff / np.sum(hist_original_diff)
    hist_generated_norm = hist_generated_diff / np.sum(hist_generated_diff)
    # Calcular distancia de Bhattacharyya
    hist_diff = np.sum(np.sqrt(hist_original_norm * hist_generated_norm))

    ## Calcular los errores de gradiente
    # Definir los kernels de Sobel para las derivadas X e Y
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    # Aplicar convolución para obtener gradientes
    grad_x_original = convolve(original_image, sobel_x)
    grad_y_original = convolve(original_image, sobel_y)
    grad_x_generated = convolve(generated_image, sobel_x)
    grad_y_generated = convolve(generated_image, sobel_y)
    # Calcular diferencia de gradientes
    grad_diff = np.mean(np.abs(grad_x_original - grad_x_generated) + np.abs(grad_y_original - grad_y_generated))

    ## Calcular IoU
    original_array = (original_array - original_array.min()) / (original_array.max() - original_array.min()) * 255
    generated_array = (generated_array - generated_array.min()) / (generated_array.max() - generated_array.min()) * 255


    original_binary = original_array > 128
    generated_binary = generated_array > 128
    
    # Calcular la intersección y la unión
    intersection = np.logical_and(original_binary, generated_binary).sum()
    union = np.logical_or(original_binary, generated_binary).sum()
    
    # Evitar divisiones por cero
    if union <= 0:
        return psnr_value, mse, mae, ssim_value, lpips_value, pearson_corr, cross_entropy, hist_diff, grad_diff, 0.0
    IoU = intersection / union

    return psnr_value, mse, mae, ssim_value, lpips_value, pearson_corr, cross_entropy, hist_diff, grad_diff, IoU

In [ ]:
psnr_value, mse, mae, ssim_value, lpips, pearson_corr, cross_entropy, hist_diff, grad_diff, IoU = calculate_metrics(original_image, generated_image)

print("PSNR:", round(psnr_value, 2))
print("Intersection Over Union:", round(IoU, 2))
print("MSE:", round(mse, 2))
print("MAE:", round(mae, 2))
print("SSIM:", round(ssim_value, 2))
print("LPIPS:", round(lpips, 2))
print("Correlacion de Pearson:", round(pearson_corr, 2))
print("Entropía cruzada:", round(cross_entropy, 2))
print("Diferencia de Histograma:", round(hist_diff, 2))
print("Errores de gradiente:", round(grad_diff, 2))

### Statistics

In [ ]:
psnr_total = 0
iou_total = 0
mse_total = 0
mae_total = 0
ssim_total = 0
lpips_total = 0
pearson_corr_total = 0
cross_entropy_total = 0
hist_diff_total = 0
grad_diff_total = 0
contador = 0

psnr_values = []
iou_values = []
mse_values = []
mae_values = []
ssim_values = []
lpips_values = []
pearson_corr_values = []
cross_entropy_values = []
hist_diff_values = []
grad_diff_values = []

for i in tqdm(range(0, 15)):
    # Comprobar que existe la imagen
    if not os.path.isfile(f"C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\results\\dualseq-bt_rg-v1_recur-2\\dualseq-bt_rg-v1_sketch\\test_latest\\images\\step{i}_generated_fake_B.png"):
        continue
        
    # Cargar imágenes
    original_image = Image.open(f"C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\results\\dualseq-bt_rg-v1\\test_latest\\images\\PANC-1_{i+1}_real_B.png").convert('L')
    generated_image = Image.open(f'C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\results\\dualseq-bt_rg-v1_recur-2\\dualseq-bt_rg-v1_sketch\\test_latest\\images\\step{i}_generated_fake_B.png').convert('L')
    
    # Convertir imágenes PIL a matrices NumPy
    original_array = np.array(original_image)
    generated_array = np.array(generated_image)

    # Ejecutar la función
    psnr_value, mse, mae, ssim_value, lpips, pearson_corr, cross_entropy, hist_diff, grad_diff, iou = calculate_metrics(original_image, generated_image)

    # Añade a la lista los valores de cada imagen
    psnr_values.append(psnr_value)
    iou_values.appdned(iou)
    mse_values.append(mse)
    mae_values.append(mae)
    ssim_values.append(ssim_value)
    lpips_values.append(lpips)
    pearson_corr_values.append(pearson_corr)
    cross_entropy_values.append(cross_entropy)
    hist_diff_values.append(hist_diff)
    grad_diff_values.append(grad_diff)

    # Number of images
    contador += 1
    
# Calcular las medias
psnr_media = sum(psnr_values) / contador
iou_media = sum(iou_values) / contador
mse_media = sum(mse_values) / contador
mae_media = sum(mae_values) / contador
ssim_media = sum(ssim_values) / contador
lpips_media = sum(lpips_values) / contador
pearson_corr_media = sum(pearson_corr_values) / contador
cross_entropy_media = sum(cross_entropy_values) / contador
hist_diff_media = sum(hist_diff_values) / contador
grad_diff_media = sum(grad_diff_values) / contador

# Calcular la desviación típica
psnr_std = np.std(psnr_values)
iou_std = np.std(iou_values)
mse_std = np.std(mse_values)
mae_std = np.std(mae_values)
ssim_std = np.std(ssim_values)
lpips_std = np.std(lpips_values)
pearson_corr_std = np.std(pearson_corr_values)
cross_entropy_std = np.std(cross_entropy_values)
hist_diff_std = np.std(hist_diff_values)
grad_diff_std = np.std(grad_diff_values)

# Métricas y valores
metricas = ['PSNR', 'IoU', 'MSE', 'MAE', 'SSIM', 'LPIPS', 'Correlacion de Pearson', 'Entropia cruzada', 'Diferencia de histograma', 'Errores de gradiente']
valores = [psnr_values, iou_values, mse_values, mae_values, ssim_values, lpips_values, 
           pearson_corr_values, cross_entropy_values, hist_diff_values, grad_diff_values]

# Medias y desviaciones estándar
medias = [psnr_media, iou_media, mse_media, mae_media, ssim_media, lpips_media, pearson_corr_media, cross_entropy_media, hist_diff_media, grad_diff_media]
desviaciones = [psnr_std, iou_std, mse_std, mae_std, ssim_std, lpips_std, pearson_corr_std, cross_entropy_std, hist_diff_std, grad_diff_std]

# Calcula el coeficiente de variación de cada métrica
coeficientes_variacion = []
for i, j in zip(medias, desviaciones):
    cv = (j / i) * 100
    coeficientes_variacion.append(cv)

# Añadir a las lista que incluye todos los valores para su posterior representación
valores.append(coeficientes_variacion)

# Imprimir resultados
print("Las métricas obtenidas de ", contador, "imágenes son:")
for i, j in zip(medias, metricas):
    i = round(i, 3)
    print(f'La media de {j} es: {i}')
    
print('-'*60)
for i, j in zip(desviaciones, metricas):
    i = round(i, 3)
    print(f'La desviación típica de {j} es: {i}')

print('-'*60)
for i, j in zip(coeficientes_variacion, metricas):
    i = round(i, 3)
    print(f'El coeficiente de variación de {j} es: {i}')

## FID

In [ ]:
def calculate_fid(num_images, path_real, path_fake):
    # Cargar modelo InceptionV3
    model = inception_v3(pretrained=True, transform_input=False)
    model.fc = torch.nn.Identity()  # Quitar la última capa para obtener características
    model.eval()

    # Transformaciones para InceptionV3
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalización Inception
    ])

    real_images = []
    fake_images = []
    # Cargar imágenes en bucle
    contador = 0
    for i in range(num_images):
        try:
            real_img = transform(Image.open(path_real.format(i)).convert('RGB'))
            fake_img = transform(Image.open(path_fake.format(i)).convert('RGB'))
            real_images.append(real_img)
            fake_images.append(fake_img)
            contador +=1
        except FileNotFoundError:
            continue
            

    # Convertir listas en tensores
    real_images = torch.stack(real_images)
    fake_images = torch.stack(fake_images)

    # Mover a GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    real_images, fake_images = real_images.to(device), fake_images.to(device)
    model.to(device)

    # Obtener características
    with torch.no_grad():
        features_real = model(real_images).cpu().numpy()
        features_fake = model(fake_images).cpu().numpy()

    # Calcular estadísticas (media y covarianza)
    mu_real, sigma_real = np.mean(features_real, axis=0), np.cov(features_real, rowvar=False)
    mu_fake, sigma_fake = np.mean(features_fake, axis=0), np.cov(features_fake, rowvar=False)

    # Calcular FID
    diff = mu_real - mu_fake
    cov_mean = sqrtm(sigma_real @ sigma_fake)

    if np.iscomplexobj(cov_mean):
        cov_mean = cov_mean.real

    fid = np.sum(diff**2) + np.trace(sigma_real + sigma_fake - 2 * cov_mean)
    return fid, contador

In [ ]:
num_images = 1000  # Número de imágenes a evaluar
base_path = "C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO4\\results\\dualseq-CSIC-top_left\\dualseq-bt_rg-v1_sketch\\test_latest\\images\\PANC-1_"
base_path2 = "C:\\Users\\Jesus\\Documents\\TFM\\PROYECTO3\\csic\\top_left\\PANC-1_"
p1 = "{}_real_B.jpg"
p2 = "{}_fake_B.png"
p3 = "{}_real_A.png"
p4 = "{}_fake_A.png"
fid_value, images = calculate_fid(num_images, 
                          base_path2 + "{}.jpg", 
                          base_path + p2)

print(f"FID: {fid_value} for {images} images.")

## Representation

In [ ]:
# Métricas y datos para valores altos
metricas_valores_altos = ['PSNR', 'MSE', 'MAE', 'Errores de gradiente']
medias_valores_altos = [psnr_media, mse_media, mae_media, grad_diff_media]
desviaciones_valores_altos = [psnr_std, mse_std, mae_std, grad_diff_std]

# Métricas y datos para valores bajos
metricas_valores_bajos = ['SSIM', 'LPIPS', 'Correlacion de Pearson', 'Entropia cruzada', 'Diferencia de histograma']
medias_valores_bajos = [ssim_media, lpips_media, pearson_corr_media, cross_entropy_media, hist_diff_media]
desviaciones_valores_bajos = [ssim_std, lpips_std, pearson_corr_std, cross_entropy_std, hist_diff_std]

# Crear la figura con dos subgráficas
fig, axs = plt.subplots(2, 1, figsize=(8, 10))

# Subgráfica para valores altos
axs[0].errorbar(metricas_valores_altos, medias_valores_altos, yerr=desviaciones_valores_altos, fmt='o', capsize=5, label='Media ± Desviación')
axs[0].set_xticklabels(metricas_valores_altos, rotation=45, ha='right')
axs[0].set_ylabel('Valor')
axs[0].set_title('Medias y Desviaciones Estándar de las Métricas con valores altos')
axs[0].legend()
axs[0].grid(True)

# Subgráfica para valores bajos
axs[1].errorbar(metricas_valores_bajos, medias_valores_bajos, yerr=desviaciones_valores_bajos, fmt='o', capsize=5, label='Media ± Desviación')
axs[1].set_xticklabels(metricas_valores_bajos, rotation=45, ha='right')
axs[1].set_ylabel('Valor')
axs[1].set_title('Medias y Desviaciones Estándar de las Métricas con valores bajos')
axs[1].legend()
axs[1].grid(True)

# Ajustar automáticamente el diseño para evitar superposiciones
plt.tight_layout()

plt.show()


In [ ]:
# Lista de valores que quieres graficar
valores_grandes = [psnr_values, mse_values, mae_values, grad_diff_values]
valores_pequeños = [ssim_values, lpips_values, pearson_corr_values, cross_entropy_values, hist_diff_values]

# Lista de etiquetas para el histograma
etiquetas_pequeños = ['SSIM', 'LPIPS', 'Correlacion de Pearson', 'Entropia cruzada', 'Diferencia de histograma']
etiquetas_grandes = ['PSNR', 'MSE', 'MAE', 'Errores de gradiente']

# Crear figuras y ejes
fig, axs = plt.subplots(1, 2, figsize=(12, 8))

# Diagrama de caja valores pequeños (Boxplot)
axs[0].boxplot(valores_pequeños)
axs[0].set_xticklabels(etiquetas_pequeños, rotation=45, ha='right', fontsize=18)
axs[0].set_title('Boxplots de métricas con valores bajos', fontsize = 20)

# Diagrama de caja valores grandes (Boxplot)
axs[1].boxplot(valores_grandes)
axs[1].set_xticklabels(etiquetas_grandes, rotation=45, ha='right', fontsize=18)
axs[1].set_title('Boxplots de metricas con valores altos', fontsize = 20)

# Ajustar diseño de las subfiguras
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [ ]:
# Crear subgráficos de histogramas para cada métrica
fig, axs = plt.subplots(3, 3, figsize=(12, 12))

# Aplanar la matriz de subgráficos para iterar fácilmente sobre ella
axs = axs.flatten()

for i, (valor, etiqueta) in enumerate(zip(valores, metricas)):
    axs[i].hist(valor, bins=20, color='skyblue', edgecolor='black')
    axs[i].set_xlabel('Magnitud', fontsize = 14)
    axs[i].set_ylabel('Valor', fontsize = 14)
    axs[i].set_title(f'Histograma de {etiqueta}', fontsize = 14)
    axs[i].grid(True)

# Ajustar automáticamente el diseño para evitar superposiciones
plt.tight_layout()
plt.show()